# 📊 Validations Transilien : décrypter la routine francilienne au 2ᵉ trimestre 2024  
## Une série temporelle racontant le pouls du transport en commun hors métro, ses week-ends et ses moments d’exception le 1ᵉʳ, 8 et 29 mai 2024  

Durant la période d’avril à juin 2024, plus de **7,2 milliards de validations** ont été recensées sur les réseaux franciliens *hors métro*. Ces données, publiées en open-data par Île-de-France Mobilités via l’URL [https://data.iledefrance-mobilites.fr](https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/validations-reseau-surface-nombre-validations-par-jour-2eme-trimestre/exports/parquet?lang=fr&timezone=Europe%2FBerlin), offrent un récit inédit du quotidien francilien. Chaque validation est un fragment d’histoire — trajets domicile-travail, escapades dominicales en tramway ou randonnées périurbaines — capturé sous la forme d’un timestamp précis.  

L’objectif de cette analyse est simple mais ambitieux : transformer ces millions de lignes en une **courbe temporelle narrative**, capable de mettre en lumière quatre régimes de circulation : semaine, week-end et trois journées d’exception majeures le 1ᵉʳ mai (Fête du Travail), le 8 mai (Commémoration 1945) et le 29 mai (Ascension). Ces jours frappent le réseau d’un silence presque rituel : chutes vertigineuses des validations, plages horaires désertes et fréquences réduites. En mettant en valeur ces ruptures sur fond de tendances saisonnières, nous voulons proposer un tableau dynamique, intuitif et exploratoire des habitudes de mobilité en région parisienne.  

---

## Méthodologie  

1. **Acquisition**  
   Les données brutes sont récupérées en format Parquet depuis l’API d’Île-de-France Mobilités puis ouvertes dans DuckDB pour leur robustesse en grands volumes.  

2. **Sélection et agrégation**  
   Une requête SQL condense la granularité « jour / ligne / titre » en volumes quotidiens globaux, enrichis de deux indicateurs booléens :  
   - `est_we` := 1 si le jour est un samedi ou un dimanche, 0 sinon ;  
   - `est_ferie` := 1 pour les dates spécifiques 2024-05-01, 2024-05-08 et 2024-05-29, 0 sinon.  

   L’utilisation de `GROUP BY ALL` garantit que la somme des validations conserve chaque jour unique, indispensable à la série temporelle.  

3. **Visualisation interactive**  
   Avec Plotly et pandas, la courbe est tracée via `go.Scatter` pour le fil principal, des `vrect` gris translucides signalent les week-ends, tandis qu’une seconde trace `go.Scatter` à marqueurs rouges met en lumière les jours fériés. Le `rangeslider` et le `rangeselector` permettent un zoom fluide.  

Cette approche combine rigueur statistique et storytelling : en un coup d’œil, nous pouvons voir combien chaque pause calendaire fait vaciller le système de transport francilien, et combien il repart aussitôt en marche.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/validations-reseau-surface-nombre-validations-par-jour-2eme-trimestre/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT
    jour as date_validation,
    SUM(nb_vald) as total_validations,
    CASE WHEN EXTRACT(dow FROM jour) IN (0,6) THEN 1 ELSE 0 END as est_we,
    CASE WHEN jour IN ('2024-05-01', '2024-05-08', '2024-05-29') THEN 1 ELSE 0 END as est_ferie
FROM loaded_dataset
GROUP BY ALL
ORDER BY jour """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

Le code mobilise Plotly, une bibliothèque interactive de graphiques web, pour exposer l’évolution des validations transport. Cette technologie permet de superposer sur la même courbe les trajets du quotidien, les week-ends grisés comme zones de référence, et les jours fériés en étoiles rouges, le tout filtrable via le curseur de plage de dates.

In [3]:
import pandas as pd
import duckdb as ddb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# S'assurer que la date est bien un datetime et trier
df['date_validation'] = pd.to_datetime(df['date_validation'])
df = df.sort_values('date_validation')

dataviz = go.Figure()

# Courbe principale des validations
dataviz.add_trace(
    go.Scatter(
        x=df['date_validation'],
        y=df['total_validations'],
        mode='lines',
        line=dict(color='#1f77b4', width=2),
        name='Validations',
        hovertemplate='%{x|%d/%m/%Y}<br>Validations: %{y:,.0f}<extra></extra>'
    )
)

# Ajout des plages grises pour les weekends
for _, row in df[df['est_we'] == 1].iterrows():
    dataviz.add_vrect(
        x0=row['date_validation'],
        x1=row['date_validation'] + pd.Timedelta(days=1),
        fillcolor='gray',
        opacity=0.2,
        layer='below',
        line_width=0,
        name='Week-end'
    )

# Marqueurs pour les jours fériés
feries = df[df['est_ferie'] == 1]
if not feries.empty:
    dataviz.add_trace(
        go.Scatter(
            x=feries['date_validation'],
            y=feries['total_validations'],
            mode='markers',
            marker=dict(
                size=10,
                color='red',
                symbol='star'
            ),
            name='Jour férié',
            hovertemplate='%{x|%d/%m/%Y}<br>Jour férié<extra></extra>'
        )
    )

dataviz.update_layout(
    title=dict(
        text="Évolution des validations de titres de transport",
        font=dict(size=18, family="Arial")
    ),
    xaxis_title="Date",
    yaxis_title="Nombre de validations",
    template='plotly_white',
    height=500,
    hovermode='x unified',
    margin=dict(l=20, r=20, t=60, b=20),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)"
    )
)

dataviz.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="7j", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

dataviz.update_yaxes(tickformat=",.0f")
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*